<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Monday 30 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 1: Staging data for analysis</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas, 
<sup>3, 4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>5 </sup>Rhys Whitley
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>4 </sup>Geography, University of Exeter, Exeter, United Kingdom
<br>
<sup>5 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to process the separate netCDF4 files for the model drivers (X<sub>i=1, 2, ... M</sub>) and model target (Y) into a unified tabular data frame, exported as a compressed comma separated value (CSV) file. This file is subsequently used in the Bayesian inference study that forms the second notebook in this experiment. The advantage of the pre-processing the data separately to the analysis allows for it be quickly staged on demand. Of course other file formats may be more advantageous for greater compression (e.g. SQLite3 database file).
</p>
<br>
<b>You will need to run this notebook to prepare the dataest before you attempt the Bayesian analysis in Part 2</b>.
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
<hr>
</font>
</center>
</div>

## Load libraries

Changed

In [33]:
# data munging and analytical libraries 
import re
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset 

# graphical libraries
import matplotlib.pyplot as plt
%matplotlib inline

# set paths
inPath = "../outputs/Australia_region/"
outPath = "../temp/SE_AUS_climate_TS/"


## Import and clean data

Set the directory path and look for all netcdf files that correspond to the model drivers and target.

In [34]:
driver_paths_all = [os.path.join(dp, f) for (dp, _, fn) in os.walk(inPath) for f in fn if f.endswith('.nc')]
driver_paths_all = [path for path in driver_paths_all if 'full_period' in  path]
driver_names_all = [re.search('^[a-zA-Z_]*', os.path.basename(fp)).group(0) for fp in driver_paths_all]

file_table = pd.DataFrame({'filepath': driver_paths_all, 'file_name': driver_names_all})

def checkFilename(driver_name, sat):
    
    if "firecount" in driver_name:
        if sat in driver_name: return True
        return False
    
    return True

#def select_files_for_sat(sat):
#    test = [checkFilename(driver_name, sat) for driver_name in driver_names_all]
    
driver_paths = np.array(driver_paths_all)#test]
driver_names = np.array(driver_names_all)#[test]
print(driver_paths)
#    driver_names = ["firecount" if "firecount" in driver_name else driver_name for driver_name in driver_names]
#    return driver_paths, driver_names
    
driver_info = [driver_paths, driver_names]#[select_files_for_sat(sat) for sat in satalites]

['../outputs/Australia_region/climate/full_period/emc-1979-2019.nc'
 '../outputs/Australia_region/climate/full_period/soilw.0-10cm.gauss.1949-2019.nc'
 '../outputs/Australia_region/vegetation/full_period/MaxOverMean_soilw.0-10cm.gauss.1949-2019.nc']


Define a function to extract the variable values from each netCDF4 file. Variables are flattened from a 3 dimensional array to 1 dimensional version, pooling all values both spatially and temporily. 

Don't know if this is the correct way to do this, but will come back to it once I understand the model (and its optimisation) better.

In [35]:
from pdb import set_trace as browser

def nc_extract(fpath, fname):
    print(fpath)
    print("Processing: {0}".format(fpath))
    with Dataset(fpath, 'r') as nc_file:
        gdata = np.array(nc_file.variables['variable'][:,:,:])
        
        gdata[gdata < -9E9] = np.nan
        def anyNaN(x):
            return np.any(np.isnan(x))
        
        if type(gdata) == np.ma.core.MaskedArray:
            browser()
        else:            
            mask = np.apply_along_axis(anyNaN, 0, gdata)
            gdata = gdata[:,~mask]
        
        
        df = pd.DataFrame(np.array(gdata))
        outPathi = outPath + fname + '.csv'
        savepath = os.path.expanduser(outPathi)
        df.to_csv(savepath, index=False)
        

Execute the above function on all netCDF4 file paths.

In [36]:
values = [nc_extract(dp, fname) for dp, fname in zip(driver_paths, driver_names)]

../outputs/Australia_region/climate/full_period/emc-1979-2019.nc
Processing: ../outputs/Australia_region/climate/full_period/emc-1979-2019.nc
../outputs/Australia_region/climate/full_period/soilw.0-10cm.gauss.1949-2019.nc
Processing: ../outputs/Australia_region/climate/full_period/soilw.0-10cm.gauss.1949-2019.nc
../outputs/Australia_region/vegetation/full_period/MaxOverMean_soilw.0-10cm.gauss.1949-2019.nc
Processing: ../outputs/Australia_region/vegetation/full_period/MaxOverMean_soilw.0-10cm.gauss.1949-2019.nc
